<a href="https://colab.research.google.com/github/SarahLares/Coursera_Capstone/blob/master/Capstone_The_Battle_of_the_Neighborhoods_for_a_Tea_Salon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>

A **tea room**, is a small establishment where light food and tea related drinks are served and offered. As of 2009, with the arrival of large tea chains, the boom of tea rooms in **Argentina **begins, the consumption of this drink has become popular and there are more entrepreneurs wanting to start a tea business. 

On the other hand, coffee shops have also become very popular in the City of **Buenos Aires**, and they are usually the main competition for tea rooms.

There are various tea franchises in Argentina, with the arrival of these new tea entrepreneurs and the coffe shop franchises, the **tea franchises have to choose strategically where to open their new stores** so that this is a success and thus not lose valuable company resources, these **franchises are the target audience**.

The main objective of this project is the **determination of the location for a new store for a tea room franchise in the city of Buenos Aires through the implementation of machine learning algorithms**.


## Data <a name="data"></a>


* List of neighborhoods in Buenos Aires, Argentina.

* Latitude and Longitude of these neighborhoods. [Data Set Here](https://data.buenosaires.gob.ar/dataset/barrios)

* Information about the business opportunities of the Buenos Aires city. [Data Set Here](https://data.buenosaires.gob.ar/dataset/mapa-oportunidades-comerciales-moc)

* Venue data related to tea rooms and coffes. This will help us find the neighborhoods that are most suitable to open a tea room. 


The data sets of the city of Buenos Aires, was obteined from the page [buenosaires.gob.ar](https://data.buenosaires.gob.ar/). This datasets are public data generated, saved and published by the Government of the City of Buenos Aires.

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import folium 
from pandas import json_normalize 
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

###Coordenate of the City of Buenos Aires

In [ ]:
address = 'Buenos Aires, Argentina'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Buenos Aires City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Buenos Aires City are -34.6075682, -58.4370894.


In [ ]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

In [168]:
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
import numpy as np

In [166]:
BA_gdf = gpd.read_file('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson')
BA_gdf['centroid_lon'] = BA_gdf['geometry'].centroid.x
BA_gdf['centroid_lat'] = BA_gdf['geometry'].centroid.y


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [185]:
BA_url_cierre = 'https://raw.githubusercontent.com/SarahLares/Coursera_Capstone/master/datasets/cierre.csv'
BA_cierre = pd.read_csv(BA_url_cierre,delimiter=';')
BA_url_apertura = 'https://raw.githubusercontent.com/SarahLares/Coursera_Capstone/master/datasets/apertura.csv'
BA_apertura = pd.read_csv(BA_url_apertura,delimiter=';')
BA_apertura['AÑO'].value_counts()

2016    3858
2017    1352
Name: AÑO, dtype: int64

In [176]:
m = folium.Map(location=[latitude,longitude],
               tiles='Stamen Toner', zoom_start=12)


# add markers with basic information
fg = folium.FeatureGroup(name='District Info')
for lat, lon, name in zip(BA_gdf['centroid_lat'].tolist(),
                               BA_gdf['centroid_lon'].tolist(),
                               BA_gdf['barrio'].tolist()):
    html = f"""
    <h2>{name}<\h2><br>
    """
    fg.add_child(folium.CircleMarker(location=[lat, lon], popup=html,radius=3,
                               color='blue',fill=True,fill_color='#3186cc',
                               fill_opacity=0.7,))

m.add_child(fg)

# enable layers to be turned in or out
folium.LayerControl().add_to(m)
m